In [22]:
import pandas as pd
import os
import geopandas as gpd
from keplergl import KeplerGl
import json

In [23]:
morbi = pd.read_csv('../data/morbilidad_Fgen.csv')

In [24]:
morbi.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
89455,23,1,23,2,F4323,1,50,55,1.00000,F43,...,2016-06-21,2016-04-27,2016,4,27,Urgente,Curación,Hombre,Jaén,Jaén
246564,8,1,8,2,F1920,1,44,1,1.09860,F19,...,2018-07-11,2018-07-10,2018,7,10,Urgente,Curación,Hombre,Barcelona,Barcelona
274351,33,1,33,2,F259,2,49,6,1.10912,F25,...,2018-05-30,2018-05-24,2018,5,24,Urgente,Traslado,Hombre,Asturias,Asturias


In [81]:
morbi.fecha_ing =  pd.to_datetime(morbi.fecha_ing)

In [40]:
total_pro = morbi.groupby('ProvHosp_nom').agg({'ProvHosp':'count'}).reset_index().rename({'ProvHosp':'ingresos'},axis=1)

In [41]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [42]:
poly = {p['properties']['name']:p['geometry'] for p in provincias['features'] }

In [43]:
total_pro['pol'] = total_pro['ProvHosp_nom'].map(poly)

In [44]:
total_pro.sample()

,ProvHosp_nom,ingresos,pol
44,Toledo,2696,"{'type': 'MultiPolygon', 'coordinates': [[[[-4..."


In [45]:
total_pob = pd.read_csv('../data/poblacion_prov.csv')

In [46]:
total_pob = total_pob[total_pob['Sexo'] == 'Total']

In [47]:
total_pob.sample(3)

,Provincias,cod_pr,Sexo,Total
92,Navarra,31,Total,607095.88
143,Vizcaya,48,Total,1144973.25
77,La Rioja,26,Total,302381.38


In [48]:
dic_pob = {}
for i,r in total_pob.iterrows():
    dic_pob[r.Provincias] = r.Total

In [49]:
total_pro['poblacion'] = total_pro.ProvHosp_nom.map(dic_pob)

In [50]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion
44,Toledo,2696,"{'type': 'MultiPolygon', 'coordinates': [[[[-4...",633374.67
4,Asturias,11076,"{'type': 'MultiPolygon', 'coordinates': [[[[-5...",1066441.50
3,Almería,3283,"{'type': 'MultiPolygon', 'coordinates': [[[[-2...",636670.54


In [60]:
total_pro['ponde'] = ((total_pro.ingresos/total_pro.poblacion)*100000).round(2)

In [61]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion,ponde
42,Tarragona,9281,"{'type': 'MultiPolygon', 'coordinates': [[[[1....",730851.79,1269.89
37,Salamanca,2467,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",346244.71,712.50
17,Girona,5983,"{'type': 'MultiPolygon', 'coordinates': [[[[1....",688770.21,868.65


In [ ]:
#Create a basemap 
map_1= KeplerGl(height=700, weight = 500, data={'total_pro': total_pro})
#show the map
map_1

In [65]:
morbi.sample()

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
137233,7,2,7,2,F200,2,58,37,1.0,F20,...,2017-11-02,2017-09-26,2017,9,26,Urgente,Traslado,Mujer,Baleares,Baleares


In [89]:
pro_day = morbi.groupby(['ProvHosp_nom','year','month']).agg({'day':'count','fecha_ing':'min'}).reset_index().rename({'day':'total'},axis=1)

In [90]:
pro_day.sample()

,ProvHosp_nom,year,month,total,fecha_ing
1501,Guipúzcoa,2019,8,135,2019-08-01


In [91]:
pro_day.shape

(3997, 5)

In [92]:
pro_day['geometry']= pro_day.ProvHosp_nom.map(poly)

In [ ]:
map_2= KeplerGl(height=700, weight = 700, data={'Ingresos_dia': pro_day})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
